In [1]:
import torch 
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable

In [2]:
# Hyper Parameters
num_epochs = 5
batch_size = 100
learning_rate = 0.001

In [3]:
# MNIST Dataset
train_dataset = dsets.MNIST(root='./data/',
                            train=True, 
                            transform=transforms.ToTensor(),
                            download=True)

test_dataset = dsets.MNIST(root='./data/',
                           train=False, 
                           transform=transforms.ToTensor())

# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size, 
                                          shuffle=False)

Processing...
Done!


In [4]:
# CNN Model (2 conv layer)
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2))
        self.fc = nn.Linear(7*7*32, 10)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out
        
cnn = CNN()

In [5]:
# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(cnn.parameters(), lr=learning_rate)

In [6]:
# Train the Model
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = Variable(images)
        labels = Variable(labels)
        
        # Forward + Backward + Optimize
        optimizer.zero_grad()
        outputs = cnn(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [%d/%d], Iter [%d/%d] Loss: %.4f' 
                   %(epoch+1, num_epochs, i+1, len(train_dataset)//batch_size, loss.data[0]))

Epoch [1/5], Iter [100/600] Loss: 0.1003
Epoch [1/5], Iter [200/600] Loss: 0.0721
Epoch [1/5], Iter [300/600] Loss: 0.0879
Epoch [1/5], Iter [400/600] Loss: 0.0343
Epoch [1/5], Iter [500/600] Loss: 0.1069
Epoch [1/5], Iter [600/600] Loss: 0.0390
Epoch [2/5], Iter [100/600] Loss: 0.0055
Epoch [2/5], Iter [200/600] Loss: 0.1176
Epoch [2/5], Iter [300/600] Loss: 0.0452
Epoch [2/5], Iter [400/600] Loss: 0.0244
Epoch [2/5], Iter [500/600] Loss: 0.0201
Epoch [2/5], Iter [600/600] Loss: 0.0170
Epoch [3/5], Iter [100/600] Loss: 0.0521
Epoch [3/5], Iter [200/600] Loss: 0.0300
Epoch [3/5], Iter [300/600] Loss: 0.0210
Epoch [3/5], Iter [400/600] Loss: 0.0479
Epoch [3/5], Iter [500/600] Loss: 0.0365
Epoch [3/5], Iter [600/600] Loss: 0.0029
Epoch [4/5], Iter [100/600] Loss: 0.0424
Epoch [4/5], Iter [200/600] Loss: 0.0476
Epoch [4/5], Iter [300/600] Loss: 0.0477
Epoch [4/5], Iter [400/600] Loss: 0.0169
Epoch [4/5], Iter [500/600] Loss: 0.0101
Epoch [4/5], Iter [600/600] Loss: 0.0721
Epoch [5/5], Ite

In [7]:
# Test the Model
cnn.eval()  # Change model to 'eval' mode (BN uses moving mean/var).
correct = 0
total = 0
for images, labels in test_loader:
    images = Variable(images)
    outputs = cnn(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum()

print('Test Accuracy of the model on the 10000 test images: %d %%' % (100 * correct / total))

Test Accuracy of the model on the 10000 test images: 99 %


In [8]:
# Save the Trained Model
torch.save(cnn.state_dict(), 'cnn.pkl')

In [9]:
cnn.state_dict()

OrderedDict([('layer1.0.weight', 
              (0 ,0 ,.,.) = 
                0.0192 -0.2377 -0.0259 -0.0034  0.2317
               -0.1871 -0.1309  0.1481 -0.0222  0.1515
               -0.1007 -0.1540 -0.2077 -0.0076 -0.1569
                0.1484 -0.0424 -0.0330 -0.0250 -0.2016
                0.0209 -0.0152 -0.0557 -0.1260  0.1461
              
              (1 ,0 ,.,.) = 
                0.1470 -0.0353  0.0420  0.0920 -0.2643
               -0.0345 -0.0829  0.2217 -0.0669 -0.2647
                0.0042 -0.0952  0.1670  0.1553 -0.0742
               -0.1442 -0.0184 -0.1295  0.2014 -0.0783
               -0.1256  0.0406 -0.0267 -0.0253  0.1131
              
              (2 ,0 ,.,.) = 
               -0.0609 -0.0292 -0.1582  0.0756 -0.1383
               -0.2166 -0.0005 -0.1600  0.1455 -0.0943
               -0.0515 -0.1928  0.1504  0.0795 -0.1611
               -0.1230 -0.1065  0.2082 -0.0167 -0.0891
               -0.2524  0.0246  0.2278 -0.1187 -0.0322
              
         